# 1. 크롤링
- 알라딘 홈페이지에서 ISBN 가져오기

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [2]:
service = Service('chromedriver.exe')
driver = webdriver.Chrome(service = service)

In [4]:
# CID : 카테고리(소설 : 1, 에세이 :55889 , 자기계발 : 336)
results = []

for page in range(651,701): # 페이지 입력
    url = "https://www.aladin.co.kr/shop/wbrowse.aspx?BrowseTarget=List&ViewRowsCount=50&ViewType=Detail\
    &PublishMonth=0&SortOrder=2&page={}&Stockstatus=0&PublishDay=84&CustReviewRankStart=0&CustReviewCountStart=0&PriceFilterMax=-1\
    &CID=1&SearchOption=".format(page)
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 페이지에 있는 도서 링크(50개)
    book_links = soup.select('ul > li > a.bo3')
    links = []
    for book_link in book_links:
        a= book_link['href']
        links.append(a)
    for link in links:
        driver.get(link)
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        try:
            ISBN = soup.select('div.conts_info_list1 > ul > li')
            ISBN = ISBN[-1].text[-13:]
        except:
            pass

        #data = [ISBN]
        results.append(ISBN)

In [4]:
results[-1]

'9788932918952'

In [6]:
import pickle
ISBN21 = pd.DataFrame(results)
ISBN21.to_pickle('ISBN21.plk')

# 2. API 조회하기
- ISBN 정보로 API 도서검색

In [5]:
import requests
import pandas as pd
import json

In [6]:
# API 호출키
KEY = '*************'

In [7]:
url_look_list = []

for isbn in results:
    url_look = 'https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey={}&itemIdType=isbn13&ItemId={}&output=JS&Version=20131101&OptResult=authors,reviewList,fulldescription,fulldescription2,Story,categoryIdList'.format(KEY, isbn)
    url_look_list.append(url_look)
print(url_look_list)

['https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey=ttbsueji951535001&itemIdType=isbn13&ItemId=9788934977155&output=JS&Version=20131101&OptResult=authors,reviewList,fulldescription,fulldescription2,Story,categoryIdList', 'https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey=ttbsueji951535001&itemIdType=isbn13&ItemId=9788965134459&output=JS&Version=20131101&OptResult=authors,reviewList,fulldescription,fulldescription2,Story,categoryIdList', 'https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey=ttbsueji951535001&itemIdType=isbn13&ItemId=9791159033483&output=JS&Version=20131101&OptResult=authors,reviewList,fulldescription,fulldescription2,Story,categoryIdList', 'https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey=ttbsueji951535001&itemIdType=isbn13&ItemId=9791187815228&output=JS&Version=20131101&OptResult=authors,reviewList,fulldescription,fulldescription2,Story,categoryIdList', 'https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey=ttbsueji951535001&itemIdType=isbn13&It

In [8]:
items = []
lists = []
for url_look in url_look_list:
    try:
        response = requests.get(url_look)
        text = json.loads(response.text)
        #time.sleep(1)
        title = text['item'][0]['title']
        author = text['item'][0]['author']
        pubDate = text['item'][0]['pubDate']
        description = text['item'][0]['description']
        isbn13 = text['item'][0]['isbn13']
        categoryID = text['item'][0]['categoryId']
        categoryName = text['item'][0]['categoryName']
        salesPoint = text['item'][0]['salesPoint']
        adult = text['item'][0]['adult']
        rank = text['item'][0]['customerReviewRank']
        fullDes = text['item'][0]['fullDescription']
        fullDes2 = text['item'][0]['fullDescription2']
        story = text['item'][0]['subInfo']['story']
        try:
            reviewrank = text['item'][0]['reviewList'][0]['reviewRank']
            reviewtitle = text['item'][0]['reviewList'][0]['title']
            review_w = text['item'][0]['reviewList'][0]['writer']
        except:
            reviewrank = 0
            reviewtitle = 0
            review_w = 0
        lists.append([title, author, pubDate, description,isbn13,categoryID,categoryName,salesPoint,adult,rank,fullDes,fullDes2,story,reviewrank,reviewtitle,review_w])
    except:
        pass
cols = ['title', 'author', 'pubDate','description','isbn13','categoryID','categoryName','salesPoint','adult','rank','fullDes','fullDes2','story','reviewrank','reviewtitle','review_w']
df = pd.DataFrame(lists, columns = cols)

In [9]:
import pickle
df.to_pickle('novel_651-700.pkl', protocol=4)

In [10]:
df

,title,author,pubDate,description,isbn13,categoryID,categoryName,salesPoint,adult,rank,fullDes,fullDes2,story,reviewrank,reviewtitle,review_w
0,벤트로드,"로리 로이 (지은이), 하현길 (옮긴이)",2017-04-14,비채 모중석스릴러클럽의 마흔두 번째 책. 그 주인공은 '미스터리 소설계의 오스카상'...,9788934977155,51062,국내도서>소설/시/희곡>추리/미스터리소설>영미 추리/미스터리소설,56,False,8,비채 모중석스릴러클럽의 마흔두 번째 책. 그 주인공은 '미스터리 소설계의 오스카상'...,<b>25년 동안 비밀에 휩싸여 있던 비극적 가족사<BR>\n마침내 드러나는 처절한...,,8,[100자평]요즘 영어권 작가는 데뷔작부터 빛나는 작가들이 참 많구...,비밀의문
1,본질에 대하여,"그레구아르 들라쿠르 (지은이), 김수진 (옮긴이)",2017-03-14,"그레구아르 들라쿠르 소설. 한 번도 자신의 진짜 모습으로 살지 못한 한 여자, 느닷...",9788965134459,50921,국내도서>소설/시/희곡>프랑스소설,56,False,8,"그레구아르 들라쿠르 소설. 한 번도 자신의 진짜 모습으로 살지 못한 한 여자, 느닷...",<b>『행복만을 보았다』 작가 그레구아르 들라쿠르의 질문<BR>\n우리 삶의 본질을...,,8,[마이리뷰]<본질에 대하여>를 읽고,강구민
2,오만과 편견 1 미니북 - 전2권 (한글판 + 영문판),"제인 오스틴 (지은이), 김유미 (옮긴이)",2017-03-01,더클래식 세계문학 컬렉션 미니북. 1813년 작품이 처음 발표된 이후 무수히 많은 ...,9791159033483,50919,국내도서>소설/시/희곡>영미소설,56,False,0,더클래식 세계문학 컬렉션 미니북. 1813년 작품이 처음 발표된 이후 무수히 많은 ...,<b>18세기 영국 낭만주의 대표 여성 작가<BR>\n제인 오스틴 최고의 걸작!<B...,베넷 가 다섯 자매의 첫째 딸인 제인은 온순하고 순진했다. 둘째 딸인 엘리자베스는 ...,0,0,0
3,원해 2,꽃쏘 (지은이),2017-02-28,"꽃쏘 장편소설. ""모델 같은 거 해 본 적은?"" ""관심 없고, 관심 없을 예정이니까...",9791187815228,51125,국내도서>소설/시/희곡>로맨스소설>한국 로맨스소설,56,False,0,"꽃쏘 장편소설. ""모델 같은 거 해 본 적은?"" ""관심 없고, 관심 없을 예정이니까...","“모델 같은 거 해 본 적은?”<BR>\n“관심 없고, 관심 없을 예정이니까 놔.”...",,0,0,0
4,이랑비랑 한약국 - 화가야 Vol.2,이영희 (지은이),2017-01-04,"이영희 장편소설. 이랑비랑 한약국, 그곳에 이 년 만에 손님이 찾아온다. 이 년 전...",9791104910647,51125,국내도서>소설/시/희곡>로맨스소설>한국 로맨스소설,56,False,8,"이영희 장편소설. 이랑비랑 한약국, 그곳에 이 년 만에 손님이 찾아온다. 이 년 전...",<b>꽃의 나라 화(花)가야!<BR>\n꽃으로 가득한 신비스러운 나라에서 펼쳐지는<...,,2,[100자평]제 취향이 아니예요. 괜히 구매햇네요.,모선영
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2439,모던 컬렉션 1~5 세트 - 전5권,"버지니아 울프, 에밀리 브론테, 제인 오스틴, 찰스 디킨스, 헤르만 헤세 (지은이)...",2015-07-23,세월의 비평을 이겨내고 수많은 이들의 마음속에 살아남은 세계의 명작들을 엄선하여 소...,9788962603644,50919,국내도서>소설/시/희곡>영미소설,47,False,0,세월의 비평을 이겨내고 수많은 이들의 마음속에 살아남은 세계의 명작들을 엄선하여 소...,세월의 비평을 이겨내고 수많은 이들의 마음속에 살아남은 세계의 명작들을 엄선하여 소...,,0,0,0
2440,담벼락 너머의 Mr.괴물 2,서향 (지은이),2015-07-22,"서향의 로맨스 소설. 스물아홉 살 어느 날, 회사가 부도처리 되면서 사장이 도주했다...",9788928061730,51125,국내도서>소설/시/희곡>로맨스소설>한국 로맨스소설,47,False,8,"서향의 로맨스 소설. 스물아홉 살 어느 날, 회사가 부도처리 되면서 사장이 도주했다...","‘먹을 게 앞에 있는데, 먹기를 포기했다니. 게다가 하의실종이었는데?’<br />발...",,8,[100자평]인간여자와 신이라는 상상속의 인물과 사랑이야기 입니다....,모선영
2441,시경의 사랑 노래 : 혼인편,"뚜안추잉 (지은이), 박종혁 (옮긴이)",2015-07-20,중국 뚜안추잉 교수가 지은 &lt;시경의 사랑노래 詩經中的情歌&gt;를 완역한 책이...,9788960715301,51168,국내도서>소설/시/희곡>시>외국시,47,False,0,중국 뚜안추잉 교수가 지은 &lt;시경의 사랑노래 詩經中的情歌&gt;를 완역한 책이...,이 책은 중국 뚜안추잉段楚英 교수가 지은 ≪시경의 사랑노래 詩經中的情歌≫를 완역한 ...,,0,0,0
2442,변방의 슈라우드 2 - Novel Engine,"미쿠모 가쿠토 (지은이), 아카시오 (그림), 정호욱 (옮긴이)",2015-07-01,『스트라이크 더 블러드』로 유명한 작가 미쿠모 가쿠토의 신작! 일본의 『전격문고 M...,9791131932513,54719,국내도서>소설/시/희곡>라이트 노벨>노블엔진(Novel Engine),47,False,8,『스트라이크 더 블러드』로 유명한 작가 미쿠모 가쿠토의 신작! 일본의 『전격문고 M...,<b>『스트라이크 더 블러드』의 미쿠모 가쿠토가 보내드리는<br />신감각 일상계 ...,,4,"[100자평]변방의 슈라우드 2 - 작위적, 너무 뻔한 클리세의 모...",로리!
